In [7]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
)

import torch.nn as nn

def has_sentences(example):
    """
    Функция возвращает True, если пример содержит хотя бы одно предложение
    в documents_sentences (и, опционально, в response_sentences).
    Иначе возвращает False.
    """

    if "documents_sentences" not in example:
        return False
    if not isinstance(example["documents_sentences"], list):
        return False
    total_doc_sentences = 0
    for doc in example["documents_sentences"]:
        if isinstance(doc, list):
            total_doc_sentences += len(doc)
    if total_doc_sentences == 0:
        return False

    if "response_sentences" in example:
        if not isinstance(example["response_sentences"], list):
            return False
        if len(example["response_sentences"]) == 0:
            return False

    return True

MODEL_NAME = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def preprocess(example, max_length=1024):
    question_ids = tokenizer.encode(example["question"], add_special_tokens=False)
    doc_ids, rel_labels, util_labels = [], [], []

    for doc in example["documents_sentences"]:
        for key, sent in doc:
            tokens = tokenizer.encode(sent, add_special_tokens=False)
            doc_ids += tokens
            rel_labels += [float(key in example["all_relevant_sentence_keys"])] * len(tokens)
            util_labels += [float(key in example["all_utilized_sentence_keys"])] * len(tokens)

    response_ids = tokenizer.encode(example["response"], add_special_tokens=False)
    adh_labels = [float(example["adherence_score"])] * len(response_ids)

    sep_id = tokenizer.sep_token_id
    input_ids = question_ids + [sep_id] + doc_ids + [sep_id] + response_ids

    context_mask = [0] * (len(question_ids)+1) + [1]*len(doc_ids) + [0] + [0]*len(response_ids)
    response_mask = [0]*(len(question_ids)+len(doc_ids)+2) + [1]*len(response_ids)

    rel_labels = [0.0]*(len(question_ids)+1) + rel_labels + [0.0]*(len(response_ids)+1)
    util_labels = [0.0]*(len(question_ids)+1) + util_labels + [0.0]*(len(response_ids)+1)
    adh_labels = [0.0]*(len(question_ids)+len(doc_ids)+2) + adh_labels

    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        context_mask = context_mask[:max_length]
        response_mask = response_mask[:max_length]
        rel_labels = rel_labels[:max_length]
        util_labels = util_labels[:max_length]
        adh_labels = adh_labels[:max_length]

    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor([1]*len(input_ids), dtype=torch.long),
        'context_mask': torch.tensor(context_mask, dtype=torch.bool),
        'response_mask': torch.tensor(response_mask, dtype=torch.bool),
        'labels_relevance': torch.tensor(rel_labels, dtype=torch.float),
        'labels_utilization': torch.tensor(util_labels, dtype=torch.float),
        'labels_adherence': torch.tensor(adh_labels, dtype=torch.float),
    }





In [8]:
from promt_test.data import load_full_ragbench

In [9]:

# Загрузка датасета
dataset = load_full_ragbench()#load_dataset("rungalileo/ragbench", "delucionqa")
dataset = {key:dataset[key].filter(has_sentences) for key in dataset}

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 314/314 [00:00<00:00, 674.99 examples/s]


In [2]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # batch: list of dicts
    keys = batch[0].keys()
    out = {}
    #  max len
    max_len = max(len(x['input_ids']) for x in batch)
#     print(batch)
    for k in keys:
        vals = [x[k] for x in batch]
        if k == 'input_ids':
            out[k] = pad_sequence(vals, batch_first=True, padding_value=tokenizer.pad_token_id)
        elif k == 'attention_mask':
            out[k] = pad_sequence(vals, batch_first=True, padding_value=0)
        elif k in ['context_mask', 'response_mask']:
            out[k] = pad_sequence(vals, batch_first=True, padding_value=False)
        elif k.startswith('labels_'):
            out[k] = pad_sequence(vals, batch_first=True, padding_value=0)
        else:
            out[k] = torch.stack(vals)
    return out
train_data = [preprocess(ex) for ex in dataset['train']]
val_data   = [preprocess(ex) for ex in dataset['validation']]
batch_size=8

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


In [3]:
dataset['train'][1].keys()

dict_keys(['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adherence_score', 'overall_supported_explanation', 'relevance_explanation', 'all_relevant_sentence_keys', 'all_utilized_sentence_keys', 'trulens_groundedness', 'trulens_context_relevance', 'ragas_faithfulness', 'ragas_context_relevance', 'gpt3_adherence', 'gpt3_context_relevance', 'gpt35_utilization', 'relevance_score', 'utilization_score', 'completeness_score'])

In [4]:
processed = preprocess(dataset['train'][1])


tokens = tokenizer.convert_ids_to_tokens(processed["input_ids"])

labels_rel = processed["labels_relevance"]
labels_util = processed["labels_utilization"]
labels_adh = processed["labels_adherence"]

print(f"{'Token':20} | {'Relevance':9} | {'Utilization':11} | {'Adherence':9}")
print("-" * 60)
i = 0
for token, rel, util, adh in zip(tokens, labels_rel, labels_util, labels_adh):
    if token == '[PAD]':
        break
    i+=1
    print(f"{token:20} | {rel:9.1f} | {util:11.1f} | {adh:9.1f}")
    if token == '▁Simply':
        print(i)

Token                | Relevance | Utilization | Adherence
------------------------------------------------------------
▁how                 |       0.0 |         0.0 |       0.0
▁to                  |       0.0 |         0.0 |       0.0
▁customize           |       0.0 |         0.0 |       0.0
▁U                   |       0.0 |         0.0 |       0.0
connect              |       0.0 |         0.0 |       0.0
▁system              |       0.0 |         0.0 |       0.0
▁based               |       0.0 |         0.0 |       0.0
▁on                  |       0.0 |         0.0 |       0.0
▁my                  |       0.0 |         0.0 |       0.0
▁own                 |       0.0 |         0.0 |       0.0
▁preferences         |       0.0 |         0.0 |       0.0
?                    |       0.0 |         0.0 |       0.0
[SEP]                |       0.0 |         0.0 |       0.0
▁U                   |       0.0 |         0.0 |       0.0
connect              |       0.0 |         0.0 |      

In [5]:
import torch.nn.functional as F
from scipy.special import expit
import numpy as np
class DebertaTrace(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model
        hid = base_model.config.hidden_size
        self.rel_head = nn.Linear(hid,1)
        self.util_head = nn.Linear(hid,1)
        self.adh_head = nn.Linear(hid,1)

    def forward(self, input_ids, attention_mask):
        out = self.base(input_ids=input_ids, attention_mask=attention_mask)
        hs = out.last_hidden_state
        return {
            'logits_relevance': self.rel_head(hs).squeeze(-1),
            'logits_utilization': self.util_head(hs).squeeze(-1),
            'logits_adherence': self.adh_head(hs).squeeze(-1)
        }


def compute_batch_metrics(logits, labels, masks):
    # logits: dict of tensors [B,L]
    # labels: dict of tensors [B,L]
    # masks: dict of bool tensors [B,L]
    rel_log, util_log, adh_log = logits['logits_relevance'], logits['logits_utilization'], logits['logits_adherence']
    rel_lab, util_lab, adh_lab = labels['labels_relevance'], labels['labels_utilization'], labels['labels_adherence']
    ctx_m, resp_m = masks['context_mask'], masks['response_mask']

    # probs
    rel_p = torch.tensor(expit(rel_log.detach().cpu().numpy()))
    util_p = torch.tensor(expit(util_log.detach().cpu().numpy()))
    adh_p = torch.tensor(expit(adh_log.detach().cpu().numpy()))
    # preds
    pred_rel = rel_p>0.5
    pred_util = util_p>0.5
    pred_adh = adh_p>0.5
    # true
    true_rel = rel_lab.detach().cpu()==1
    true_util = util_lab.detach().cpu()==1
    true_adh = adh_lab.detach().cpu()==1

    # per example
    def ex_metric(pred, true, mask):
        inter = (pred & true & mask).sum(dim=1).float()
        denom = (true & mask).sum(dim=1).float()
        return (inter/denom.clamp(min=1)).mean().item()

    rel_m = ex_metric(pred_rel, true_rel, ctx_m)
    util_m= ex_metric(pred_util, true_util, ctx_m)
    adh_m = ex_metric(pred_adh, true_adh, resp_m)
    comp = ((pred_rel & pred_util & ctx_m).sum(dim=1).float()/ (true_rel & ctx_m).sum(dim=1).float().clamp(min=1)).mean().item()
    return rel_m, util_m, adh_m, comp

In [6]:
def check_for_nans(logits, labels, mask, head_name=""):
    """
    Проверяет тензоры logits и labels для данного сегмента mask на наличие NaN.
    Если NaN найдены, выводит подробную информацию и выбрасывает ошибку.
    """
    # Извлечение элементов по маске
    logit_vals = logits[mask]
    label_vals = labels[mask]
    
    # Проверка на NaN
    nan_logits = torch.isnan(logit_vals)
    nan_labels = torch.isnan(label_vals)
    
    if nan_logits.any() or nan_labels.any():
        print(f"NaN detected in head '{head_name}':")
        if nan_logits.any():
            idxs = torch.nonzero(nan_logits, as_tuple=True)[0]
            print(f"  logits NaN at positions: {idxs.tolist()}")
            print(f"  sample logits: {logit_vals[idxs].tolist()}")
        if nan_labels.any():
            idxs = torch.nonzero(nan_labels, as_tuple=True)[0]
            print(f"  labels NaN at positions: {idxs.tolist()}")
            print(f"  sample labels: {label_vals[idxs].tolist()}")
        # Остановим обучение для отладки
        raise ValueError(f"NaN encountered in head '{head_name}'")

In [7]:
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base = AutoModel.from_pretrained(MODEL_NAME)
model=DebertaTrace(base).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=2e-5)
scaler = torch.amp.GradScaler('cuda')
accum=4

for epoch in range(5):
    model.train()
    total_loss=0
    opt.zero_grad()
    loop = tqdm(train_loader, desc=f"Training Epoch {epoch}", total=len(train_loader))
    for i, batch in enumerate(loop, 1):
        inputs = {'input_ids': batch['input_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}
        labels = {k: v.to(device) for k,v in batch.items() if k.startswith('labels_')}
        masks  = {k: v.to(device) for k,v in batch.items() if k.endswith('_mask')}
        with torch.amp.autocast('cuda'):
            out = model(**inputs)
            
            l_rel = F.binary_cross_entropy_with_logits(out['logits_relevance'][masks['context_mask']], labels['labels_relevance'][masks['context_mask']])
            check_for_nans(
                out['logits_relevance'], labels['labels_relevance'],
                masks['context_mask'], head_name="relevance"
            )

            l_util= F.binary_cross_entropy_with_logits(out['logits_utilization'][masks['context_mask']], labels['labels_utilization'][masks['context_mask']])
            check_for_nans(
                out['logits_utilization'], labels['labels_utilization'],
                masks['context_mask'], head_name="labels_utilization"
            )

            l_adh = F.binary_cross_entropy_with_logits(out['logits_adherence'][masks['response_mask']], labels['labels_adherence'][masks['response_mask']])
            check_for_nans(
                out['logits_adherence'], labels['labels_adherence'],
                masks['response_mask'], head_name="labels_adherence"
            )
            loss = (l_rel + l_util + l_adh)/ 3 #/ accum
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#         if (i+1)%accum==0:
        scaler.step(opt)
        scaler.update()
        opt.zero_grad()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    print(f"Epoch {epoch} loss: {total_loss:.4f}")

    # валидация
    model.eval()
    sums = np.zeros(4);
    count=0
    
    with torch.no_grad():
        
        for batch in tqdm(val_loader):
            inputs = {'input_ids': batch['input_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}
            labels = {k: v for k,v in batch.items() if k.startswith('labels_')}
            masks  = {k: v for k,v in batch.items() if k.endswith('_mask')}
            out = model(**{k:inputs[k] for k in inputs})
            m = compute_batch_metrics(out, labels, masks)
            sums += np.array(m)
            count+=1
    avg = sums/count
    print(f"Eval -> relevance={avg[0]:.4f}, utilization={avg[1]:.4f}, adherence={avg[2]:.4f}, completeness={avg[3]:.4f}")


Training Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [02:23<00:00,  1.27it/s, loss=0.311]


Epoch 0 loss: 82.8243


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:07<00:00,  3.26it/s]


Eval -> relevance=0.2904, utilization=0.2807, adherence=0.6703, completeness=0.7012


Training Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [02:23<00:00,  1.28it/s, loss=0.784]


...

Eval -> relevance=0.3739, utilization=0.3623, adherence=0.6703, completeness=0.9031


Training Epoch 4: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [02:23<00:00,  1.27it/s, loss=0.225]


Epoch 4 loss: 58.0869


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:07<00:00,  3.27it/s]

Eval -> relevance=0.4787, utilization=0.4599, adherence=0.6702, completeness=0.9939
